# Project Outline

This project looks at the stock price of Roku. By doing a time series analysis and also a sentiment analysis on social media for the stock price of Roku we will try and predict future prices.

Project flow:
 * Gather stock price
 * Gather posts from twitter and reddit
 * Do a sentiment analysis for each day
 * Do a time series analysis on both stock price and sentiment individually
 * Combine the sentiment and stock price to try and predict future prices

# Packages used

In [1]:
# Installing important packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import twint
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from psaw import PushshiftAPI
import praw
import datetime as dt
import yfinance as yf
import requests 
import requests.auth
from Config import *
import time

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


# This is so twint will be able to work
import nest_asyncio
nest_asyncio.apply()

# Gathering stock data

In [2]:
stock_data = yf.download('ROKU','2017-09-28','2021-05-31')

[*********************100%***********************]  1 of 1 completed


In [14]:
# Turning the stock data index into a list we can itterate through
stock_days = stock_data.index

# Making list to itterate through for Reddit Data
stock_days_central = stock_days.tz_localize(tz='US/Central')
stock_days_itter = [int(x.timestamp()) for x in stock_days_central]

# Making list to itterate through for twitter data
twitter_stock_days = stock_data.index.strftime('%Y-%m-%d')

In [4]:
local_datetime_converted = dt.datetime.fromtimestamp(stock_days_itter[0])
print(local_datetime_converted)

2017-09-28 00:00:00


# Gathering social media data

## Twitter Data

### Twitter Data Test

#### CSV route

In [ ]:
# # Gathering twitter data

# # Configure
# config = twint.Config()
# config.Search = "$ROKU"
# config.Since = '2021-05-09'
# config.Until = '2021-05-11'
# config.Lang = 'en'
# config.Store_csv = True
# config.Min_likes = 5
# config.Limit = 20
# config.Popular_tweets = True
# config.Output = '/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone'

# # Running query search
# twint.run.Search(config)


In [ ]:
# test = pd.read_csv('tweets.csv')
# test.dropna(axis=1, inplace=True)
# test.drop_duplicates(inplace=True)
# test

In [ ]:
# test.columns

#### Pandas route

In [25]:
# Configure
c = twint.Config()
c.Search = '$ROKU'
c.Since = '2021-05-05'
c.Until = '2021-05-07'
c.Lang = 'en'
c.Limit = 20  #Has to be increments of 20
c.Pandas = True
c.Pandas_clean = True
c.Pandas_au = True
c.Popular_tweets = True
c.Min_likes = 10

# Running the search
twint.run.Search(c)

# Storing to dataframe
Tweets_df = twint.storage.panda.Tweets_df

# Dropping unwanted columns
Tweets_df.drop(columns=['conversation_id','created_at','place','hashtags','user_id',
                       'user_id_str','name','link','urls','photos','video','thumbnail','retweet',
                       'quote_url','near','geo','source','user_rt_id','user_rt','retweet_id','reply_to',
                       'retweet_date','translate','trans_src','trans_dest'], inplace=True)

1390048334119833601 2021-05-05 15:58:43 -0500 <ParrotStock> Whelp, Twitter is cooperating about as well as my brokerage today 😂   Going to try and post this for the 3rd time today...  No moves and trying to ignore the ST noise 😊  Despite the effort, the reds prevailed $PTON $ROKU $ATER $MGNI $IDN $SHOP $CRWD $DMTK $SKLZ  Another day 🦜  https://t.co/0lOr4sHw1l
1390047541891264516 2021-05-05 15:55:34 -0500 <StockDweebs> Day 2: Trying to get to 6/7 - figures  Starting Balance: $50,000 Unrealized P&amp;L: -$431.18 Realized P&amp;L: $0.00  Current Positions: $ROKU, $NVDA Trade Ideas: $TWTR, $BABA, $NFLX, $SQ, $SAVE, $ATVI, $SHAK  Comments, portfolio, and spreadsheets screenshots attached.  https://t.co/XMhEjSPMAc
1390029715566649346 2021-05-05 14:44:44 -0500 <StockMKTNewz> Top 10 holdings of Cathie Wood &amp; Ark Invest's $ARKK Ark Innovation ETF as of today 5/5  1 Tesla $TSLA 2 Teladoc $TDOC 3 $ROKU 4 Square $SQ 5 Shopify $SHOP 6 Zillow $Z $ZG 7 Zoom $ZM 8 Spotify $SPOT 9 Twilio $TWLO 10 E

In [26]:
Tweets_df

,id,date,timezone,tweet,language,cashtags,username,day,hour,nlikes,nreplies,nretweets,search
0,1390048334119833601,2021-05-05 15:58:43,-0500,"Whelp, Twitter is cooperating about as well as...",en,"[pton, roku, ater, mgni, idn, shop, crwd, dmtk...",ParrotStock,3,15,60,25,2,$ROKU
1,1390047541891264516,2021-05-05 15:55:34,-0500,Day 2: Trying to get to 6/7 - figures Startin...,en,"[roku, nvda, twtr, baba, nflx, sq, save, atvi,...",StockDweebs,3,15,253,33,6,$ROKU
2,1390029715566649346,2021-05-05 14:44:44,-0500,Top 10 holdings of Cathie Wood &amp; Ark Inves...,en,"[arkk, tsla, tdoc, roku, sq, shop, z, zg, zm, ...",StockMKTNewz,3,14,45,6,3,$ROKU
3,1390019124571500544,2021-05-05 14:02:39,-0500,$roku down 20% in 4 days on notbing,en,[roku],Crussian17,3,14,86,29,1,$ROKU
4,1390008384133337090,2021-05-05 13:19:58,-0500,If $roku drops tmrw on earnings im doubling do...,en,[roku],Crussian17,3,13,57,12,1,$ROKU
5,1389991777151569924,2021-05-05 12:13:59,-0500,Bought $ROKU 15OCT21 350.00c small. This is m...,en,[roku],data168,3,12,37,4,0,$ROKU
6,1389940123953991685,2021-05-05 08:48:44,-0500,@phoenixvalue $ROKU,und,[roku],WallSt_Dropout,3,08,11,1,0,$ROKU
7,1389939852368560132,2021-05-05 08:47:39,-0500,"Right now, $CMCSA is valued at 6X $ROKU... No...",en,"[cmcsa, roku, roku]",anandchokkavelu,3,08,38,4,1,$ROKU
8,1389927011745996804,2021-05-05 07:56:37,-0500,GM Stratters! Up and down we go. I prefer the ...,en,"[fdx, ma, zm, team, roku, pypl, nvda, okta, etsy]",r3dpepsi,3,07,24,4,7,$ROKU
9,1389913397026770944,2021-05-05 07:02:31,-0500,"Yellen's Error, 'Inflation Scare', Cyclical Se...",en,"[aapl, roku, snap, dash, rtx, lmt, amat, amd, ...",Sarge986,3,07,17,2,4,$ROKU


In [27]:
Tweets_df.columns

Index(['id', 'date', 'timezone', 'tweet', 'language', 'cashtags', 'username',
       'day', 'hour', 'nlikes', 'nreplies', 'nretweets', 'search'],
      dtype='object')

In [33]:
Tweets_df.iloc[1]['tweet']

'Day 2: Trying to get to 6/7 - figures  Starting Balance: $50,000 Unrealized P&amp;L: -$431.18 Realized P&amp;L: $0.00  Current Positions: $ROKU, $NVDA Trade Ideas: $TWTR, $BABA, $NFLX, $SQ, $SAVE, $ATVI, $SHAK  Comments, portfolio, and spreadsheets screenshots attached.  https://t.co/XMhEjSPMAc'

### Looping through and collecting Twitter data

In [ ]:
for i in range(len(twitter_stock_days)):
   
    # Configure
    c = twint.Config()
    c.Search = '$ROKU'
    
    ### Change since and until
    if i < 922:
        c.Since = twitter_stock_days[i]
        next_day = i+1
        c.Until = twitter_stock_days[next_day]
    
    elif i == 922:
        c.Since = twitter_stock_days[i]
        c.Until = '2021-05-31'
        
    c.Lang = 'en'
    c.Limit = 100  #Has to be increments of 20
    c.Pandas = True
    c.Pandas_clean = True
    c.Pandas_au = True
    c.Popular_tweets = True
    c.Min_likes = 10

    # Running the search
    twint.run.Search(c)

    # Storing to dataframe
    Tweets_df = twint.storage.panda.Tweets_df

    # Dropping unwanted columns
    Tweets_df.drop(columns=['conversation_id','created_at','place','hashtags','user_id',
                           'user_id_str','name','link','urls','photos','video','thumbnail','retweet',
                           'quote_url','near','geo','source','user_rt_id','user_rt','retweet_id','reply_to',
                           'retweet_date','translate','trans_src','trans_dest'], inplace=True)
    
    Tweets_df.sort_values(by='nlikes', axis=0, ascending=False, inplace=True)
    
    # Take the top 5 in dataframe to write
    df_to_write = Tweets_df[0:5]
    
    # Writting to a csv
    if i == 0:
        dataframe_to_write.to_csv(path_or_buf='/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/TwitterData.csv',index=False)
    elif i > 0:
        dataframe_to_write.to_csv('/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/TwitterData.csv', mode='a', header=False, index=False)
    
    time.sleep(3)

## Reddit Data

In [5]:
Reddit_subreddits = ['stocks', 'investing', 'stockmarket', 'economy', 'wallstreetbets', 'options', 'Daytrading']

### Reddit Data Test

#### Psaw/Praw

In [6]:
r = praw.Reddit(client_id=client_id,
                client_secret=client_secret,
                user_agent=user_agent)

api = PushshiftAPI(r)

In [7]:
start_epoch=int(dt.datetime(2020, 1, 1).timestamp())
end_epoch=int(dt.datetime(2020, 2, 20).timestamp())

api_request_generator = api.search_submissions(subreddit=Reddit_subreddits,
                                               after = start_epoch, before=end_epoch,
                                               q='(ROKU)|(Roku)|(roku)|(#Roku)|(#ROKU)|(#roku)|($ROKU)|($Roku)|($roku)',
                                               limit=1000)


In [8]:
Reddit_df = pd.DataFrame(columns =['ID','Num_Comments','Score','Subreddit','Title','Upvote_Ratio','Created',
                                   'Created_utc','Self_text'] )
ID=[]
Num_Comments=[]
Score=[]
Subreddit = []
Title = []
Upvote_Ratio = []
Created = []
Created_utc = []
Self_text = []

# Add day for the posts in dataframe

for submissions in api_request_generator:
    ID.append(submissions.id)
    Num_Comments.append(submissions.num_comments)
    Score.append(submissions.score)
    Subreddit.append(submissions.subreddit)
    Title.append(submissions.title)
    Upvote_Ratio.append(submissions.upvote_ratio)
    Created.append(submissions.created)
    Created_utc.append(submissions.created_utc)
    Self_text.append(submissions.selftext)
    
temp_df = pd.DataFrame({'ID':ID,
                        'Num_Comments':Num_Comments,
                        'Score':Score,
                        'Subreddit':Subreddit,
                        'Title':Title,
                        'Upvote_Ratio':Upvote_Ratio,
                        'Created':Created,
                        'Created_utc':Created_utc,
                        'Self_text':Self_text})

Reddit_df = Reddit_df.append(temp_df)
Reddit_df.sort_values(by='Score', axis=0, ascending=False, inplace=True)
Reddit_df.drop(Reddit_df[(Reddit_df['Self_text']=='[deleted]')|(Reddit_df['Self_text']=='[removed]')].index, 
              inplace=True)
Reddit_df.drop(Reddit_df[Reddit_df['Score']<21].index, inplace=True)
dataframe_to_write = Reddit_df[0:5]
dataframe_to_write.to_csv('/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/redditdata', mode='a', header=False, index=False)

In [103]:
Reddit_df

,ID,Num_Comments,Score,Subreddit,Title,Upvote_Ratio,Created,Created_utc,Self_text
118,eq0c8i,211,1141,wallstreetbets,WSB stock mentions over the last 7 days,0.98,1.579296e+09,1.579268e+09,See you in Callhalla Brothers\n\n1 SPY - 3634 ...
65,f2idft,178,705,wallstreetbets,I crashed MSFT,0.96,1.581497e+09,1.581468e+09,I’m so sorry everyone. \n\nI watched from the ...
143,ejaqzf,117,561,wallstreetbets,"Literally the first time I buy spy calls, We g...",0.98,1.578057e+09,1.578029e+09,"I just wanna say, I'm sorry. \n\nI thought ""H..."
98,exr15m,95,346,wallstreetbets,I decided to go ahead with my Hong Kong trip t...,0.99,1.580690e+09,1.580662e+09,American refuses to give me a cash refund or c...
86,f0gt6n,15995,261,wallstreetbets,"Weekend Discussion Thread - February 7-9, 2020",0.94,1.581138e+09,1.581109e+09,Welcome back to another edition of the Weekend...
124,eo6jpw,33,226,wallstreetbets,CNBC slowly succumbing to the group they resen...,0.97,1.578962e+09,1.578934e+09,
133,ems7oz,92,201,wallstreetbets,Imagine still being bullish on Roku,0.94,1.578698e+09,1.578669e+09,Imagine making so much money on the dip last y...
13,f4rfh0,131,105,wallstreetbets,Weekly Theta Gang Thread,0.95,1.581893e+09,1.581864e+09,First week where everything didn’t go perfectl...
94,eyr5mu,14,81,wallstreetbets,WARNING - Coronavirus surpasses mentions of Au...,0.95,1.580857e+09,1.580828e+09,"7 Day chatter\n\nPosts: 1,349 \nComments: 168..."
97,ey71ww,10,73,StockMarket,"Today's Pre-Market Movers & News [Monday, Febr...",0.95,1.580765e+09,1.580736e+09,#Good morning traders and investors of the r/S...


In [100]:
Reddit_df.iloc[2]['Title']

'The Inverse WSB Algorithm - 72.5% win rate - 40 day backtest'

In [101]:
Reddit_df.iloc[2]['Self_text']

'[deleted]'

In [67]:
local_datetime_converted = dt.datetime.fromtimestamp(Reddit_df.iloc[-1]['Created_utc'])
print(local_datetime_converted)

2020-01-01 10:51:13


In [20]:
Reddit_df[0:5]

,ID,Num_Comments,Score,Subreddit,Title,Upvote_Ratio,Created,Created_utc,Self_text
118,eq0c8i,211,1137,wallstreetbets,WSB stock mentions over the last 7 days,0.98,1.579296e+09,1.579268e+09,See you in Callhalla Brothers\n\n1 SPY - 3634 ...
65,f2idft,178,705,wallstreetbets,I crashed MSFT,0.96,1.581497e+09,1.581468e+09,I’m so sorry everyone. \n\nI watched from the ...
143,ejaqzf,117,561,wallstreetbets,"Literally the first time I buy spy calls, We g...",0.98,1.578057e+09,1.578029e+09,"I just wanna say, I'm sorry. \n\nI thought ""H..."
98,exr15m,95,338,wallstreetbets,I decided to go ahead with my Hong Kong trip t...,0.99,1.580690e+09,1.580662e+09,American refuses to give me a cash refund or c...
86,f0gt6n,15995,261,wallstreetbets,"Weekend Discussion Thread - February 7-9, 2020",0.94,1.581138e+09,1.581109e+09,Welcome back to another edition of the Weekend...


In [9]:
#Possible data categories that can be gathered through psaw/praw

# Index(['all_awardings', 'allow_live_comments', 'author',
#        'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
#        'author_flair_type', 'author_fullname', 'author_patreon_flair',
#        'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
#        'created_utc', 'domain', 'edited', 'full_link', 'gilded', 'gildings',
#        'id', 'is_crosspostable', 'is_meta', 'is_original_content',
#        'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
#        'link_flair_background_color', 'link_flair_css_class',
#        'link_flair_richtext', 'link_flair_template_id', 'link_flair_text',
#        'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
#        'no_follow', 'num_comments', 'num_crossposts', 'over_18',
#        'parent_whitelist_status', 'permalink', 'pinned', 'post_hint',
#        'preview', 'pwls', 'retrieved_on', 'score', 'selftext', 'send_replies',
#        'spoiler', 'stickied', 'subreddit', 'subreddit_id',
#        'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title',
#        'total_awards_received', 'treatment_tags', 'upvote_ratio', 'url',
#        'whitelist_status', 'wls', 'created'],
#       dtype='object')

### Looping through and gathering all Reddit Data

In [17]:
for i in range(len(stock_days_itter)):
    
    r = praw.Reddit(client_id=client_id,
                client_secret=client_secret,
                user_agent=user_agent)

    api = PushshiftAPI(r)
    
    start_epoch = stock_days_itter[i]
    
    if i < 922:
        next_day = i+1
        end_epoch = stock_days_itter[next_day]
    elif i == 922:
        end_epoch = int(dt.datetime(2021, 5, 29).timestamp())

    api_request_generator = api.search_submissions(subreddit=Reddit_subreddits,
                                               after = start_epoch, before=end_epoch,
                                               q='(ROKU)|(Roku)|(roku)|(#Roku)|(#ROKU)|(#roku)|($ROKU)|($Roku)|($roku)',
                                               limit=1000)
    
    Reddit_df = pd.DataFrame(columns =['ID','Num_Comments','Score','Subreddit','Title','Upvote_Ratio','Created',
                                   'Created_utc','Self_text','Date'] )
    ID=[]
    Num_Comments=[]
    Score=[]
    Subreddit = []
    Title = []
    Upvote_Ratio = []
    Created = []
    Created_utc = []
    Self_text = []
    Date = []
    

    for submissions in api_request_generator:
        ID.append(submissions.id)
        Num_Comments.append(submissions.num_comments)
        Score.append(submissions.score)
        Subreddit.append(submissions.subreddit)
        Title.append(submissions.title)
        Upvote_Ratio.append(submissions.upvote_ratio)
        Created.append(submissions.created)
        Created_utc.append(submissions.created_utc)
        Self_text.append(submissions.selftext)
        Date.append(stock_days[i])
    
    temp_df = pd.DataFrame({'ID':ID,
                            'Num_Comments':Num_Comments,
                            'Score':Score,
                            'Subreddit':Subreddit,
                             'Title':Title,
                            'Upvote_Ratio':Upvote_Ratio,
                            'Created':Created,
                            'Created_utc':Created_utc,
                            'Self_text':Self_text,
                            'Date':Date})

    Reddit_df = Reddit_df.append(temp_df)
    Reddit_df.sort_values(by='Score', axis=0, ascending=False, inplace=True)
    Reddit_df.drop(Reddit_df[(Reddit_df['Self_text']=='[deleted]')|(Reddit_df['Self_text']=='[removed]')].index, 
                  inplace=True)
    Reddit_df.drop(Reddit_df[Reddit_df['Score']<10].index, inplace=True)
    dataframe_to_write = Reddit_df[0:5]
    
    if i == 0:
        dataframe_to_write.to_csv(path_or_buf='/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/redditdata.csv',index=False)
    elif i > 0:
        dataframe_to_write.to_csv('/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/redditdata.csv', mode='a', header=False, index=False)
    
    time.sleep(3)

In [13]:
stock_days_itter[922]

1622178000

## Checking spacy sentiment analysis

In [2]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
doc = nlp(text)
doc._.polarity      # Polarity: -0.125
doc._.subjectivity  # Sujectivity: 0.9
doc._.assessments  

[(['really', 'horrible'], -1.0, 1.0, None),
 (['worst', '!'], -1.0, 1.0, None),
 (['really', 'good'], 0.7, 0.6000000000000001, None),
 (['happy'], 0.8, 1.0, None)]

# Messing with how to write to a csv file using pandas

In [110]:
Reddit_df.to_csv(path_or_buf='/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/redditdata',index=False)

In [18]:
tester = pd.read_csv('redditdata.csv')

In [24]:
tester.iloc[-10:]

,ID,Num_Comments,Score,Subreddit,Title,Upvote_Ratio,Created,Created_utc,Self_text,Date
608,n6otb0,27,37,stocks,"$ROKU Crushes Earnings, Profit and Revenue Beat",0.91,1.620385e+09,1.620357e+09,Roku reported earnings of 54 cents a share for...,2021-05-06
609,n6bafa,6,17,options,Delta-Neutral Trading: A Trial Run on UBER,0.86,1.620347e+09,1.620319e+09,Given the volatility around earnings this past...,2021-05-06
610,n78n3g,4,35,stocks,"Here is a Market Recap for today Friday, May 7...",0.87,1.620449e+09,1.620421e+09,"**PsychoMarket Recap - Friday, May 7, 2021**\n...",2021-05-07
611,n76xh3,41,28,wallstreetbets,$VZIO multi-bagger DD,0.78,1.620445e+09,1.620416e+09,"Alright dingleberries, listen up because I’m g...",2021-05-07
612,n78nm1,3,10,StockMarket,"Here is a Market Recap for today Friday, May 7...",0.87,1.620449e+09,1.620421e+09,"**PsychoMarket Recap - Friday, May 7, 2021**\n...",2021-05-07
613,na5elg,66,123,wallstreetbets,"$FUBO - Archegos, Misreported Earnings, and th...",0.92,1.620790e+09,1.620761e+09,Reporting AH ( 45 mins ) so i dont have much t...,2021-05-11
614,nah0aw,85,19,investing,Best Dip Stocks to buy right now,0.67,1.620825e+09,1.620796e+09,"Hey y'all,\n\nI have alot of cash lying around...",2021-05-12
615,nbg5ei,5,50,Daytrading,"The Only Watch List You Need May 13, 2021",0.93,1.620940e+09,1.620911e+09,*The original Gap Watch List Poster! The one t...,2021-05-13
616,nh8f2j,55,73,wallstreetbets,PUBM DD #1,0.91,1.621566e+09,1.621537e+09,\n\nA little bit about myself for context:\n\...,2021-05-20
617,nkqj4f,286,3321,stocks,Amazon’s ad revenue is now twice as big as Sna...,0.97,1.621980e+09,1.621951e+09,https://www.cnbc.com/2021/05/25/amazon-ad-reve...,2021-05-25


In [118]:
Reddit_df.to_csv('/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone/redditdata', mode='a', header=False, index=False)

In [117]:
len(Reddit_df)

72